In [1]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = np.asarray(X_train, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.int32).flatten()
X_test = np.asarray(X_test, dtype=np.float32)
y_test = np.asarray(y_test, dtype=np.int32).flatten()

num_training=49000
num_validation=1000
num_test=10000


mask = range(num_training, num_training + num_validation)
X_val = X_train[mask]
y_val = y_train[mask]
mask = range(num_training)
X_train = X_train[mask]
y_train = y_train[mask]
mask = range(num_test)
X_test = X_test[mask]
y_test = y_test[mask]

#print(X_train[0].shape)

mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
X_train = (X_train - mean_pixel) / std_pixel
X_val = (X_val - mean_pixel) / std_pixel
X_test = (X_test - mean_pixel) / std_pixel

In [2]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(64)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64)

In [20]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation = None)
        self.relu1 = tf.keras.layers.ReLU()
        self.drop1 = tf.keras.layers.Dropout(rate = 0.5)
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation = None)
        self.relu2 = tf.keras.layers.ReLU()
        self.drop2 = tf.keras.layers.Dropout(rate = 0.5)
        self.conv3 = tf.keras.layers.Conv2D(128, 3, activation = None)
        self.relu3 = tf.keras.layers.ReLU()
        self.drop3 = tf.keras.layers.Dropout(rate = 0.5)

        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(64, activation = 'relu')
        self.fc2 = tf.keras.layers.Dense(64, activation = 'relu')
        self.fc3 = tf.keras.layers.Dense(10)
        
    def call(self, x):
        x = self.conv1(x)
        #x = self.bn1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.conv2(x)
        #x = self.bn2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.drop3(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)
        
model = MyModel()

In [21]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = MyModel()
    
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer = tf.keras.optimizers.Adam(),
                 metrics=['accuracy'])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [24]:
import os

checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")


In [27]:
# Function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
    if epoch < 3:
        return 1e-3
    elif epoch >= 3 and epoch < 7:
        return 1e-4
    else:
        return 1e-5

In [28]:
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))

In [29]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [47]:
model.fit(train_ds, epochs=15, callbacks=callbacks)

Epoch 1/15
766/766 [==============================] - ETA: 0s - loss: 0.9782 - accuracy: 0.6545
Learning rate for epoch 1 is 0.0010000000474974513
766/766 [==============================] - 519s 678ms/step - loss: 0.9782 - accuracy: 0.6545 - lr: 0.0010
Epoch 2/15
766/766 [==============================] - ETA: 0s - loss: 0.8953 - accuracy: 0.6846
Learning rate for epoch 2 is 0.0010000000474974513
766/766 [==============================] - 455s 595ms/step - loss: 0.8953 - accuracy: 0.6846 - lr: 0.0010
Epoch 3/15
766/766 [==============================] - ETA: 0s - loss: 0.8266 - accuracy: 0.7093
Learning rate for epoch 3 is 0.0010000000474974513
766/766 [==============================] - 446s 583ms/step - loss: 0.8266 - accuracy: 0.7093 - lr: 0.0010
Epoch 4/15
766/766 [==============================] - ETA: 0s - loss: 0.6708 - accuracy: 0.7646
Learning rate for epoch 4 is 9.999999747378752e-05
766/766 [==============================] - 447s 583ms/step - loss: 0.6708 - accuracy: 0.7646 -

In [36]:
!dir {'training_checkpoints'}

 Volume in drive D has no label.
 Volume Serial Number is 5E15-AB74

 Directory of D:\Behnam\Uni\Videos and lectures\Machine Learning\github\Deep Learning\Tensorflow\Cifar10-TF\training_checkpoints

04/03/2020  07:24 PM    <DIR>          .
04/03/2020  07:24 PM    <DIR>          ..
04/03/2020  07:24 PM                69 checkpoint
04/03/2020  07:16 PM        67,634,721 ckpt_1.data-00000-of-00001
04/03/2020  07:16 PM             2,860 ckpt_1.index
04/03/2020  07:24 PM        67,634,721 ckpt_2.data-00000-of-00001
04/03/2020  07:24 PM             2,860 ckpt_2.index
               5 File(s)    135,275,231 bytes
               2 Dir(s)  389,232,144,384 bytes free


In [33]:
model.summary()

Model: "my_model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           multiple                  896       
_________________________________________________________________
re_lu_27 (ReLU)              multiple                  0         
_________________________________________________________________
dropout_27 (Dropout)         multiple                  0         
_________________________________________________________________
conv2d_28 (Conv2D)           multiple                  18496     
_________________________________________________________________
re_lu_28 (ReLU)              multiple                  0         
_________________________________________________________________
dropout_28 (Dropout)         multiple                  0         
_________________________________________________________________
conv2d_29 (Conv2D)           multiple                  7

In [48]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(val_ds)
print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

16/16 [==============================] - 2s 95ms/step - loss: 0.8564 - accuracy: 0.7240
Eval loss: 0.8563759922981262, Eval Accuracy: 0.7239999771118164


In [39]:
!tensorboard --logdir='./logs'

Traceback (most recent call last):
  File "c:\users\mafakheri\appdata\local\continuum\anaconda3\envs\tf_20_env\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\mafakheri\appdata\local\continuum\anaconda3\envs\tf_20_env\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\mafakheri\AppData\Local\Continuum\anaconda3\envs\tf_20_env\Scripts\tensorboard.exe\__main__.py", line 7, in <module>
  File "c:\users\mafakheri\appdata\local\continuum\anaconda3\envs\tf_20_env\lib\site-packages\tensorboard\main.py", line 75, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "c:\users\mafakheri\appdata\local\continuum\anaconda3\envs\tf_20_env\lib\site-packages\absl\app.py", line 299, in run
    _run_main(main, args)
  File "c:\users\mafakheri\appdata\local\continuum\anaconda3\envs\tf_20_env\lib\site-packages\absl\app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "c:\users\mafakheri\app

In [42]:
path = 'saved_model/'

In [43]:
model.save(path, save_format='tf')

INFO:tensorflow:Assets written to: saved_model/assets


In [44]:
unreplicated_model = tf.keras.models.load_model(path)

unreplicated_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(val_ds)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

16/16 [==============================] - 2s 101ms/step - loss: 1.0475 - accuracy: 0.6430
Eval loss: 1.047545075416565, Eval Accuracy: 0.6430000066757202


In [46]:
with strategy.scope():
    replicated_model = tf.keras.models.load_model(path)
    replicated_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy'])
    
    eval_loss, eval_acc = replicated_model.evaluate(val_ds)
    print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

16/16 [==============================] - 1s 92ms/step - loss: 1.0475 - accuracy: 0.6430
Eval loss: 1.047545075416565, Eval Accuracy: 0.6430000066757202
